In [19]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

import warnings

In [20]:
warnings.filterwarnings('ignore')

In [21]:
storm_outages = pd.read_parquet('../data/interim/storm_outages_2014_2023.parquet')

In [22]:
# Corroborar que solo haya dos valores
storm_outages.groupby('episode_fips_id').storm_caused_outage.mean().value_counts()

storm_caused_outage
0.0    388113
1.0     24346
Name: count, dtype: int64

In [23]:
episode_fips_id_value_counts = storm_outages.episode_fips_id.value_counts()
episode_fips_id_value_counts_gt1 = episode_fips_id_value_counts[episode_fips_id_value_counts > 1]

In [24]:
storm_outages[storm_outages.episode_fips_id.isin(episode_fips_id_value_counts_gt1.index)].storm_caused_outage.unique()
# Una misma tormeta puede causar varios outages en el mismo county los cuales no están relacionados entre sí. 
storm_outages[storm_outages.episode_fips_id=='107733_06037'].head()

,EPISODE_ID,fips_code_id,episode_description,begin_datetime,end_datetime,storm_duration,episode_fips_id,storm_caused_outage,outage_index_id,outage_start_minus_storm_start,outage_end_minus_storm_end,outage_start_minus_storm_end,outage_end_minus_storm_start,outage_duration,run_start_time_min,run_start_time_max
92952,107733,06037,Due to very dry vegetation and continued hot a...,2016-07-09 11:04:00,2016-07-31 23:59:00,540.916667,107733_06037,1.0,06037__0227,1.007639,-21.124306,-21.530556,1.413889,0.406250,2016-07-10 11:15:00,2016-07-10 21:00:00
92953,107733,06037,Due to very dry vegetation and continued hot a...,2016-07-09 11:04:00,2016-07-31 23:59:00,540.916667,107733_06037,1.0,06037__0228,3.413889,-19.113889,-19.124306,3.424306,0.010417,2016-07-12 21:00:00,2016-07-12 21:15:00
92954,107733,06037,Due to very dry vegetation and continued hot a...,2016-07-09 11:04:00,2016-07-31 23:59:00,540.916667,107733_06037,1.0,06037__0229,4.257639,-18.270139,-18.280556,4.268056,0.010417,2016-07-13 17:15:00,2016-07-13 17:30:00
92955,107733,06037,Due to very dry vegetation and continued hot a...,2016-07-09 11:04:00,2016-07-31 23:59:00,540.916667,107733_06037,1.0,06037__0230,6.278472,-15.457639,-16.259722,7.080556,0.802083,2016-07-15 17:45:00,2016-07-16 13:00:00
92956,107733,06037,Due to very dry vegetation and continued hot a...,2016-07-09 11:04:00,2016-07-31 23:59:00,540.916667,107733_06037,1.0,06037__0231,8.382639,-13.790972,-14.155556,8.747222,0.364583,2016-07-17 20:15:00,2016-07-18 05:00:00


In [25]:
# Simplification, we weill keep the smallest outage_date, and the longest outage_date (all the sepparated outage will be considered as just one big outage.)
storm_outages_g = storm_outages.groupby('episode_fips_id').agg(
    EPISODE_ID=('EPISODE_ID','first'),
    fips_code_id=('fips_code_id','first'),
    episode_description=('episode_description','first'),
    begin_datetime=('begin_datetime','first'),
    end_datetime=('end_datetime','first'),
    storm_duration=('storm_duration','first'),
    #episode_fips_id=('episode_fips_id','first'),
    storm_caused_outage=('storm_caused_outage','max'),
    outage_index_id=('outage_index_id','first'),
    outage_duration =('outage_duration', 'sum'),
    run_start_time_min=('run_start_time_min','min'),
    run_start_time_max=('run_start_time_max','max'),
).reset_index()


In [26]:
storm_outages_g

,episode_fips_id,EPISODE_ID,fips_code_id,episode_description,begin_datetime,end_datetime,storm_duration,storm_caused_outage,outage_index_id,outage_duration,run_start_time_min,run_start_time_max
0,100001_51139,100001,51139,High pressure sprawled over the Mid-Atlantic r...,2015-09-22 23:35:00,2015-09-23 08:00:00,8.416667,0.0,None,0.0,NaT,NaT
1,100001_51165,100001,51165,High pressure sprawled over the Mid-Atlantic r...,2015-09-22 23:35:00,2015-09-23 08:00:00,8.416667,0.0,None,0.0,NaT,NaT
2,100001_51171,100001,51171,High pressure sprawled over the Mid-Atlantic r...,2015-09-22 23:35:00,2015-09-23 08:00:00,8.416667,0.0,None,0.0,NaT,NaT
3,100001_51187,100001,51187,High pressure sprawled over the Mid-Atlantic r...,2015-09-22 23:35:00,2015-09-23 08:00:00,8.416667,0.0,None,0.0,NaT,NaT
4,100002_54023,100002,54023,High pressure sprawled over the Mid-Atlantic r...,2015-09-24 04:30:00,2015-09-24 09:35:00,5.083333,0.0,None,0.0,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
412454,99997_88538,99997,88538,Scattered showers and isolated thunderstorms f...,2015-08-11 14:12:00,2015-08-11 17:30:00,3.300000,0.0,None,0.0,NaT,NaT
412455,99998_88536,99998,88536,Strong moisture advection over the Mid-Atlanti...,2015-08-20 19:34:00,2015-08-20 21:36:00,2.033333,0.0,None,0.0,NaT,NaT
412456,99998_88537,99998,88537,Strong moisture advection over the Mid-Atlanti...,2015-08-20 19:34:00,2015-08-20 21:36:00,2.033333,0.0,None,0.0,NaT,NaT
412457,99999_88531,99999,88531,"A cold front moved through the Mid-Atlantic, t...",2015-08-24 18:49:00,2015-08-24 19:51:00,1.033333,0.0,None,0.0,NaT,NaT


In [35]:
meteorological_path = '../data/raw/meteorological'
meteorological_files = os.listdir(meteorological_path)

In [36]:
len(meteorological_files)

20569

In [37]:
def get_data_with_response_variable(met_file):
    try:
    #if True:
        #met_file = meteorological_files[0]
        episode_fips_id = met_file.split('.')[0]
        #  Read data
        path = os.path.join(meteorological_path, met_file)
        
        with open(path, 'r') as f:
            data = json.load(f)
        # Make a dataframe from information
        data = pd.DataFrame(data.get('properties').get('parameter'))
        data.index.rename('time', inplace=True)
        data.reset_index(inplace=True)
        data['episode_fips_id'] = episode_fips_id
        data['meteorological_current_datetime_val'] = pd.to_datetime(
            data['time'].apply(lambda x: x[0:4]) + 
            '-' + 
            data['time'].apply(lambda x: x[4:6]) +
            '-' + 
            data['time'].apply(lambda x: x[6:8]) +
            ' ' +
            data['time'].apply(lambda x: x[8:10]) +
            ':00:00'
        )
        # Modification to change the hour treshold.
        data['meteorological_nextHour_datetime_val'] = data['meteorological_current_datetime_val'] + pd.Timedelta(value=1, unit='hours')
        # Get the response var dataframe
        storm_outage_episode = storm_outages_g[storm_outages_g.episode_fips_id == episode_fips_id]
        meaning_dict = {
            'begin_datetime': 'storm_start', 
            'end_datetime': 'storm_end', 
            'run_start_time_min': 'outage_start', 
            'run_start_time_max': 'outage_end'
        }
        storm_outage_episode.rename(columns=meaning_dict, inplace=True)
        storm_outage_episode_columns = [
            'episode_fips_id', 
            'storm_start', 
            #'storm_end',
            'outage_start',
            'outage_end'
        ]
        # Join information
        data_rv = data.merge(
            storm_outage_episode[storm_outage_episode_columns],
            on='episode_fips_id',
            how='left'
        )
        #data_rv['outage_start_rounded'] = data_rv.outage_start.dt.round('H')
        #data_rv['outage_end_rounded'] = data_rv.outage_end.dt.round('H')
        
        # Buscamos informacion meteorologica antes del inicio del outage, después ya no nos sirve. 
        data_rv = data_rv[data_rv.meteorological_current_datetime_val <= data_rv.outage_start]
        #data_rv = data_rv[data_rv.meteorological_nextHour_datetime_val <= data_rv.outage_start_rounded]
        
        # Queremos 3 horas de info antes del inicio de la tormenta. 
        data_rv = data_rv[data_rv.meteorological_current_datetime_val >= data_rv.storm_start - pd.Timedelta(value=3, unit='hours')]

        data_rv['hours_to_outage'] = (data_rv.outage_start - data_rv.meteorological_current_datetime_val).dt.total_seconds()/3600
        data_rv['outage_in_an_hour'] = (data_rv['hours_to_outage']<=1).astype(int)
        
        #data_rv['test_val'] = data_rv.outage_start - data_rv.meteorological_current_datetime_val
        #data_rv.loc[data_rv.meteorological_nextHour_datetime_val == data_rv.outage_start_rounded, 'outage_in_an_hour'] = 1
        #data_rv.loc[data_rv.meteorological_nextHour_datetime_val != data_rv.outage_start_rounded, 'outage_in_an_hour'] = 0
        data_rv['episode_fips_time_id'] = data_rv.episode_fips_id + '_' + data_rv.time.astype(str)
        data_rv.set_index('episode_fips_time_id', inplace=True)
        return data_rv
    except:
        return None


In [38]:
#data_test, response_test = get_data_with_response_variable('164119_06029.json')

In [39]:
#response_test

In [40]:
data_rvs = []
for met_file in meteorological_files:
    data_rvs.append(get_data_with_response_variable(met_file))

In [41]:
len(data_rvs)

20569

In [42]:
cleaned_data_rvs = [x for x in data_rvs if x is not None]

In [43]:
all_data = pd.concat(cleaned_data_rvs)

In [44]:
#plt.hist(testing_prueba.time, bins=100)

In [45]:
all_data.outage_in_an_hour.sum()

np.int64(25625)

In [46]:
all_data.groupby('episode_fips_id').outage_in_an_hour.sum().sort_values()

episode_fips_id
153159_37163    1
154810_34021    1
154819_34023    1
182432_05051    1
182432_05029    1
               ..
155388_06037    2
155377_51059    2
155373_53041    2
155465_48133    2
99992_55105     2
Name: outage_in_an_hour, Length: 20569, dtype: int64

In [47]:
all_data[all_data.episode_fips_id=='128129_45045']

,time,T2M,ALLSKY_SFC_SW_DWN,PS,WS50M,ALLSKY_SFC_LW_DWN,WD50M,PRECTOTCORR,GWETPROF,CLRSKY_SFC_SW_DWN,...,CLRSKY_SFC_LW_DWN,DISPH,episode_fips_id,meteorological_current_datetime_val,meteorological_nextHour_datetime_val,storm_start,outage_start,outage_end,hours_to_outage,outage_in_an_hour
episode_fips_time_id,,,,,,,,,,,,,,,,,,,,,
128129_45045_2018061612,2018061612,29.09,920.38,96.33,1.46,430.58,146.7,1.07,0.77,969.92,...,418.35,6.33,128129_45045,2018-06-16 12:00:00,2018-06-16 13:00:00,2018-06-16 14:42:00,2018-06-16 22:00:00,2018-06-16 22:45:00,10.0,0
128129_45045_2018061613,2018061613,28.72,855.65,96.32,1.89,440.60,156.3,1.97,0.77,938.22,...,420.62,6.33,128129_45045,2018-06-16 13:00:00,2018-06-16 14:00:00,2018-06-16 14:42:00,2018-06-16 22:00:00,2018-06-16 22:45:00,9.0,0
128129_45045_2018061614,2018061614,28.23,715.35,96.32,2.12,438.02,161.0,3.34,0.77,851.35,...,421.42,6.33,128129_45045,2018-06-16 14:00:00,2018-06-16 15:00:00,2018-06-16 14:42:00,2018-06-16 22:00:00,2018-06-16 22:45:00,8.0,0
128129_45045_2018061615,2018061615,27.78,543.58,96.32,2.04,439.02,163.8,4.60,0.77,713.85,...,420.95,6.33,128129_45045,2018-06-16 15:00:00,2018-06-16 16:00:00,2018-06-16 14:42:00,2018-06-16 22:00:00,2018-06-16 22:45:00,7.0,0
128129_45045_2018061616,2018061616,27.89,421.38,96.32,1.88,430.73,165.2,4.57,0.77,539.45,...,414.92,6.33,128129_45045,2018-06-16 16:00:00,2018-06-16 17:00:00,2018-06-16 14:42:00,2018-06-16 22:00:00,2018-06-16 22:45:00,6.0,0
128129_45045_2018061617,2018061617,27.84,265.80,96.33,1.87,426.52,165.7,5.47,0.77,341.60,...,411.30,6.33,128129_45045,2018-06-16 17:00:00,2018-06-16 18:00:00,2018-06-16 14:42:00,2018-06-16 22:00:00,2018-06-16 22:45:00,5.0,0
128129_45045_2018061618,2018061618,26.63,99.70,96.35,2.00,422.10,160.1,5.30,0.77,149.38,...,402.30,6.33,128129_45045,2018-06-16 18:00:00,2018-06-16 19:00:00,2018-06-16 14:42:00,2018-06-16 22:00:00,2018-06-16 22:45:00,4.0,0
128129_45045_2018061619,2018061619,24.79,11.40,96.39,2.36,409.75,154.6,6.02,0.77,15.57,...,395.67,6.33,128129_45045,2018-06-16 19:00:00,2018-06-16 20:00:00,2018-06-16 14:42:00,2018-06-16 22:00:00,2018-06-16 22:45:00,3.0,0
128129_45045_2018061620,2018061620,23.67,0.00,96.42,2.80,410.95,156.0,6.96,0.77,0.00,...,392.48,6.33,128129_45045,2018-06-16 20:00:00,2018-06-16 21:00:00,2018-06-16 14:42:00,2018-06-16 22:00:00,2018-06-16 22:45:00,2.0,0


In [48]:
all_data.outage_in_an_hour.sum()

np.int64(25625)

In [49]:
all_data.outage_in_an_hour.mean()

np.float64(0.027549734662457963)

In [50]:
all_data.outage_in_an_hour.mean()

np.float64(0.027549734662457963)

In [51]:
pd.set_option('display.max_columns', 100)
all_data

,time,T2M,ALLSKY_SFC_SW_DWN,PS,WS50M,ALLSKY_SFC_LW_DWN,WD50M,PRECTOTCORR,GWETPROF,CLRSKY_SFC_SW_DWN,QV10M,RHOA,T10M,TO3,TQV,Z0M,TOA_SW_DWN,RH2M,WS2M,CLRSKY_SFC_LW_DWN,DISPH,episode_fips_id,meteorological_current_datetime_val,meteorological_nextHour_datetime_val,storm_start,outage_start,outage_end,hours_to_outage,outage_in_an_hour
episode_fips_time_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
179102_12011_2023032804,2023032804,24.26,0.00,101.63,2.69,376.17,191.8,5.04,0.43,0.00,16.16,1.18,24.86,256.50,40.96,0.03,0.00,87.11,1.65,373.15,0.15,179102_12011,2023-03-28 04:00:00,2023-03-28 05:00:00,2023-03-28 07:00:00,2023-03-28 21:15:00,2023-03-28 21:30:00,17.25,0
179102_12011_2023032805,2023032805,24.12,0.00,101.66,2.64,379.55,197.9,3.05,0.43,0.00,15.91,1.18,24.75,256.55,40.52,0.03,0.00,86.49,1.62,371.77,0.15,179102_12011,2023-03-28 05:00:00,2023-03-28 06:00:00,2023-03-28 07:00:00,2023-03-28 21:15:00,2023-03-28 21:30:00,16.25,0
179102_12011_2023032806,2023032806,24.21,20.12,101.71,2.58,375.55,204.8,2.20,0.43,20.75,15.85,1.18,24.70,256.22,40.22,0.03,66.88,86.38,1.53,373.12,0.15,179102_12011,2023-03-28 06:00:00,2023-03-28 07:00:00,2023-03-28 07:00:00,2023-03-28 21:15:00,2023-03-28 21:30:00,15.25,0
179102_12011_2023032807,2023032807,25.26,146.57,101.77,2.60,405.75,205.3,4.97,0.43,200.82,15.89,1.18,25.17,256.14,39.66,0.03,362.02,82.33,1.90,380.50,0.15,179102_12011,2023-03-28 07:00:00,2023-03-28 08:00:00,2023-03-28 07:00:00,2023-03-28 21:15:00,2023-03-28 21:30:00,14.25,0
179102_12011_2023032808,2023032808,26.14,347.25,101.82,2.47,407.10,197.7,2.01,0.43,427.90,15.44,1.18,25.81,256.38,38.96,0.03,654.60,75.96,2.01,385.33,0.15,179102_12011,2023-03-28 08:00:00,2023-03-28 09:00:00,2023-03-28 07:00:00,2023-03-28 21:15:00,2023-03-28 21:30:00,13.25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120432_12021_2017093001,2017093001,24.08,0.00,101.19,2.05,427.02,227.8,9.72,0.65,0.00,16.58,1.17,24.69,261.48,55.23,0.05,0.00,89.65,0.81,409.95,0.24,120432_12021,2017-09-30 01:00:00,2017-09-30 02:00:00,2017-09-29 17:00:00,2017-09-30 05:30:00,2017-09-30 07:45:00,4.50,0
120432_12021_2017093002,2017093002,23.95,0.00,101.18,2.04,427.52,215.2,7.76,0.65,0.00,16.35,1.17,24.62,262.53,54.85,0.05,0.00,88.84,0.84,409.17,0.24,120432_12021,2017-09-30 02:00:00,2017-09-30 03:00:00,2017-09-29 17:00:00,2017-09-30 05:30:00,2017-09-30 07:45:00,3.50,0
120432_12021_2017093003,2017093003,23.88,0.00,101.19,1.95,427.27,197.9,4.30,0.65,0.00,16.22,1.17,24.58,263.61,54.68,0.05,0.00,88.41,0.87,408.58,0.24,120432_12021,2017-09-30 03:00:00,2017-09-30 04:00:00,2017-09-29 17:00:00,2017-09-30 05:30:00,2017-09-30 07:45:00,2.50,0


In [52]:
sample_val = all_data[all_data.outage_in_an_hour == 1].sample(1).iloc[0]
print(sample_val.meteorological_current_datetime_val, sample_val.outage_start)
storm_outages_g[storm_outages_g.episode_fips_id == sample_val.episode_fips_id]

2020-12-17 11:00:00 2020-12-17 12:00:00


,episode_fips_id,EPISODE_ID,fips_code_id,episode_description,begin_datetime,end_datetime,storm_duration,storm_caused_outage,outage_index_id,outage_duration,run_start_time_min,run_start_time_max
188977,155048_36119,155048,36119,Low pressure developed along the Middle Atlant...,2020-12-16 15:00:00,2020-12-17 16:00:00,25.0,1.0,36119__0035,0.010417,2020-12-17 12:00:00,2020-12-17 12:15:00


In [25]:
all_data.drop(
    ['meteorological_nextHour_datetime_val', 
     'storm_start', 'outage_start', 'outage_end',], axis=1
).to_parquet('../data/interim/meteorological_data_with_outages.parquet')